# Basic CNN Model

## (1) Data Processing

- Num: 100
- Map: 직선 도로 (센서 2, 4, 6, 8번 위치)
- 처음 데이터 받을 때 모든 센서와 pop의 데이터를 붙여 버리면 시작, 끝 부분에서 이상하게 데이터가 만들어질 수 있으므로
각각 8x8xn 이미지를 만든 후에 해당 데이터를 랜덤하게 섞어서 사용한다.

In [3]:
import numpy as np
import pickle
import os

os.getcwd()

'/home/chominyong/sovico_180624'

In [4]:
data_100={}
for x in range(4):
    loc = "/home/chominyong/sovico_180624/8frame/SensorData/sensorData_" + str(2*(x+1)) + ".txt"
    print(loc)
    data_100["data_100_{0}".format(x)] = np.loadtxt(loc,delimiter=',')
    print("data_100_{0}".format(x), data_100["data_100_{0}".format(x)].shape)

/home/chominyong/sovico_180624/8frame/SensorData/sensorData_2.txt
data_100_0 (160000, 9)
/home/chominyong/sovico_180624/8frame/SensorData/sensorData_4.txt
data_100_1 (160000, 9)
/home/chominyong/sovico_180624/8frame/SensorData/sensorData_6.txt
data_100_2 (160000, 9)
/home/chominyong/sovico_180624/8frame/SensorData/sensorData_8.txt
data_100_3 (160000, 9)


In [5]:
X_100 = {}
Y_100 = {}

for x in range(4):
    split_data = np.hsplit(data_100["data_100_{0}".format(x)], np.array([1, 9]))
    Y_100["Y_100_{0}".format(x)] = split_data[0]
    X_100["X_100_{0}".format(x)] = split_data[1]

### 40x8 데이터로 만들기

In [6]:
data_length_100 = len(X_100["X_100_0"])
print("data_length_100", data_length_100)

data_length_100 160000


In [7]:
NUM_FREQ = 40

In [8]:
t = np.zeros((data_length_100-NUM_FREQ+1, NUM_FREQ, 8))
t.shape

(159961, 40, 8)

#### - Y Label 지정 ( 가장 최신 )

In [9]:
X_100_image = {}
Y_100_recent = {}
#Y_100_mid = {}

for t in range(4):
    X_100_image[t] = np.zeros((data_length_100-NUM_FREQ+1, NUM_FREQ, 8))
    Y_100_recent[t] = np.zeros(data_length_100-NUM_FREQ+1)
#    Y_100_mid[t] = np.zeros(data_length_100-7)
    for i in range(data_length_100 - NUM_FREQ+1):
        for j in range(NUM_FREQ):
            X_100_image[t][i][j][:] = X_100["X_100_{0}".format(t)][i+j][:]
           
        if Y_100["Y_100_{0}".format(t)][i+j] >= 9:
            Y_100_recent[t][i] = 9
        else:
            Y_100_recent[t][i] = Y_100["Y_100_{0}".format(t)][i+j]
        
#         if Y_100["Y_100_{0}".format(t)][i+3] >= 9:
#             Y_100_mid[t][i] = 9
#         else:
#             Y_100_mid[t][i] = Y_100["Y_100_{0}".format(t)][i+3]


        


In [10]:
X_100_image[0].shape

(159961, 40, 8)

#### 40 by 8 by 1 data

In [11]:
X_100_values = np.array(list(X_100_image.values()))
Y_100_values = np.array(list(Y_100_recent.values()))
print(X_100_values.shape)
print(Y_100_values.shape)
#values

(4, 159961, 40, 8)
(4, 159961)


In [ ]:
t = X_100_values.shape
N = t[0]*t[1]
X_modified_100_8_1 = np.zeros((N, t[2], t[3]))
Y_100_8_1 = np.zeros((N))
for i in range(t[0]):
    for j in range(t[1]):
        X_modified_100_8_1[i*t[1]+j] = X_100_values[i][j]
        Y_100_8_1[i*t[1]+j] = Y_100_values[i][j]

In [ ]:
Y_4 = np.zeros(N)

for i in range(N):
    if Y_100_8_1[i] == 0 :
        Y_4[i] = 0
    elif Y_100_8_1[i] >= 1 and Y_100_8_1[i] <= 3 :
        Y_4[i] = 1
    elif Y_100_8_1[i] >= 4 and Y_100_8_1[i] <= 6 :
        Y_4[i] = 2
    elif Y_100_8_1[i] >= 7:
        Y_4[i] = 3

print(max(Y_4))

3.0


In [ ]:
print(X_modified_100_8_1.shape)

(639844, 40, 8)


In [ ]:
for i in range(4):
    print("4 labels:",i, sum( Y_4== i ))

4 labels: 0 443695
4 labels: 1 147174
4 labels: 2 43532
4 labels: 3 5443


In [ ]:
what_is_the_least_frequency_label=3

In [ ]:
min_label_count_4 = sum(Y_4==3)
print(min_label_count_4)

5443


In [ ]:
X_balanced_4 = X_modified_100_8_1[Y_4==0][:min_label_count_4]
Y_balanced_4 = Y_4[Y_4==0][:min_label_count_4]
for i in range(1,4):    
    X_balanced_4 = np.concatenate((X_balanced_4,X_modified_100_8_1[Y_4==i][:min_label_count_4]))
    Y_balanced_4 = np.concatenate((Y_balanced_4,Y_4[Y_4==i][:min_label_count_4]))
print(X_balanced_4.shape)
print(Y_balanced_4.shape)

(21772, 40, 8)
(21772,)


In [ ]:
a = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

np.random.seed(0)
print(np.random.permutation(a))

[ 3  9  5 10  2  7  8  4  1  6]


In [ ]:
np.random.seed(0)
X_balanced_4 = np.random.permutation(X_balanced_4)
np.random.seed(0)
Y_balanced_4 = np.random.permutation(Y_balanced_4)

In [ ]:
Y_balanced_4 = np.eye(4)[Y_balanced_4.astype(int)]
Y_balanced_4

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [ ]:
Y_balanced_4.shape

(21772, 4)

In [ ]:
NN=X_balanced_4.shape[0]
num_train_balanced_4 = int(NN * .8)
print("total N, num_train",NN, num_train_balanced_4)

X_balanced_4_train = X_balanced_4[:num_train_balanced_4]
Y_balanced_4_train = Y_balanced_4[:num_train_balanced_4].astype(int)
X_balanced_4_test = X_balanced_4[num_train_balanced_4:]
Y_balanced_4_test = Y_balanced_4[num_train_balanced_4:].astype(int)
print("X_train", X_balanced_4_train.shape)
print("X_val", X_balanced_4_test.shape)
print("Y_train", Y_balanced_4_train.shape)
print("Y_val", Y_balanced_4_test.shape)

total N, num_train 21772 17417
X_train (17417, 40, 8)
X_val (4355, 40, 8)
Y_train (17417, 4)
Y_val (4355, 4)


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import keras.backend.tensorflow_backend as K

timesteps = 40
data_dim = 8
num_classes = 4

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_balanced_4_train, Y_balanced_4_train, epochs=5, batch_size=32)
model.evaluate(X_balanced_4_test, Y_balanced_4_test, batch_size=128)

Using TensorFlow backend.


Epoch 1/5
17417/17417 [==============================] - 113s 6ms/step - loss: 0.7216 - acc: 0.6828
Epoch 2/5
17417/17417 [==============================] - 111s 6ms/step - loss: 0.5323 - acc: 0.7845
Epoch 3/5
17417/17417 [==============================] - 111s 6ms/step - loss: 0.5055 - acc: 0.7936
Epoch 4/5
17417/17417 [==============================] - 111s 6ms/step - loss: 0.4878 - acc: 0.8040
Epoch 5/5
13952/17417 [=======================>......] - ETA: 21s - loss: 0.4698 - acc: 0.8121

In [ ]:
model.fit(X_balanced_4_train, Y_balanced_4_train, epochs=5, batch_size=32)
model.evaluate(X_balanced_4_test, Y_balanced_4_test, batch_size=128)

In [ ]:
model.fit(X_balanced_4_train, Y_balanced_4_train, epochs=2, batch_size=32)
model.evaluate(X_balanced_4_test, Y_balanced_4_test, batch_size=128)

In [ ]:
model.fit(X_balanced_4_train, Y_balanced_4_train, epochs=2, batch_size=32)
model.evaluate(X_balanced_4_test, Y_balanced_4_test, batch_size=128)

In [ ]:
model.fit(X_balanced_4_train, Y_balanced_4_train, epochs=2, batch_size=32)
model.evaluate(X_balanced_4_test, Y_balanced_4_test, batch_size=128)

In [ ]:
# with open('X_sim.pickle', 'rb') as handle:
#     X_test = pickle.load(handle)
# with open('Y_sim.pickle', 'rb') as handle:
#     Y_test = pickle.load(handle)

# model.evaluate(X_test, Y_test, batch_size=128)